In [1]:
!pip install matplotlib watchdog

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


In [ ]:
def read_metrics(filename):
    metrics_dict = {}
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            for part in parts:
                key, value = part.split(':')
                metrics_dict[key] = int(value)
    return metrics_dict


def update_graph(ax, metric_name, metric_values):
    ax.clear()
    ax.plot(metric_values, marker='o')
    ax.set_title(metric_name)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.grid()



In [ ]:
# Initialize the figure and subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
ax_empty_reads, ax_non_empty_reads, ax_range_queries, ax_write_queries = axs.flatten()

# Initialize empty lists to store metric values
empty_reads_values = []
non_empty_reads_values = []
range_queries_values = []
write_queries_values = []

# Initialize a variable to keep track of the last read position
last_read_position = 0

def on_metrics_file_change(event):
    global last_read_position

    with open('performance_metrics.csv', 'a') as file:
        pass 

    with open('performance_metrics.csv', 'r') as file:
        file.seek(last_read_position)
        
        for line in file:
            parts = line.strip().split(',')
            for part in parts:
                key, value = part.split(':')
                if key == 'Empty Reads Duration':
                    empty_reads_values.append(int(value))
                elif key == 'Read Duration':
                    non_empty_reads_values.append(int(value))
                elif key == 'Range Reads Duration':
                    range_queries_values.append(int(value))
                elif key == 'Write Duration':
                    write_queries_values.append(int(value))

        last_read_position = file.tell()

    # Update the graphs
    update_graph(ax_empty_reads, 'Empty Reads', empty_reads_values)
    update_graph(ax_non_empty_reads, 'Non-Empty Reads', non_empty_reads_values)
    update_graph(ax_range_queries, 'Range Queries', range_queries_values)
    update_graph(ax_write_queries, 'Write Queries', write_queries_values)


# Watch the metrics file for changes
event_handler = FileSystemEventHandler()
event_handler.on_modified = on_metrics_file_change
observer = Observer()
observer.schedule(event_handler, '.', recursive=False)
observer.start()

try:
    ani = FuncAnimation(fig, lambda x: None, interval=1000)
    plt.show()
except KeyboardInterrupt:
    observer.stop()
observer.join()
